TRAVAUX PRATIQUES DE CLASSIFICATION NON-SUPERVISEE
==============================================

Ces travaux pratiques ont pour but d’appliquer et de manipuler les différents algorithmes de classification vus en cours de de classification non supervisée. Le cas d’application est l’identification des régions homogènes en terme de température, à l’échelle climatique. Pour cela, nous disposons de 10 ans de mesures (de 1990 à 2000) journalières des température minimales (Tn) et maximales (Tx) sur 83 stations réparties sur la France métropolitaine.

Enseignant : Thomas Rieutord (thomas.rieutord@meteo.fr)

Travail demandé :
-----------------
En suivant le sujet de TP qui vous a été donné, compléter les cellules vides de ce notebook, aux endroits marqués par ###### afin de répondre aux questions.
A la fin du TP, vous devriez pouvoir exécuter l'ensemble du programme en redémarrant et exécutant le notebook : `Noyau > Redémarrer & tout exécuter`.
Vous pouvez aussi exporter le notebook en programme Python, lequel devrait tourner en tapant la commande `python3 TP_classification_IENM2.py`.

Questions préliminaires
------------------------
  1. Que va-t-on classer ? Quels sont les prédicteurs ?
  

  2. En déduire le nombre N d’individus et le nombre p de prédicteurs.

  3. Doit-on normaliser les données ? Si oui, comment ? Si non, pourquoi ?

Extraction et mise en forme des données
-------------------------------
Le fichier `../data/TN-TX-83stations.txt` contient les Tn et Tx de 83 stations météo stockées sous la forme suivante :
```
  DATE STATION1-TN STATION1-TX STATION2-TN STATION2-TX ... STATION83-TN STATION83-TX
  jour1   valeurTN  valeurTX ... valeurTN  valeurTX 
  ...
```

In [ ]:
import os
import time
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import coupdepouce as cp

In [ ]:
dataFile="../data/TN-TX-83stations_points.txt"

### Extraction
Ouvrir le fichier avec un éditeur de texte. Repérer les éléments séparateurs, la taille de l’en-tête et la nature des données (réel, entiers…).

En déduire la commande pour importer les données

In [ ]:
# Avec Numpy
alldata_np = ######
print("Retourne un ",type(alldata_np)," de taille ",np.shape(alldata_np))

# Avec Pandas
alldata_pd = ######
print("Retourne un ",type(alldata_pd)," de taille ",np.shape(alldata_pd))

print("\nComparaison ?\t alldata_pd.values est un ",type(alldata_pd.values)," de taille ",np.shape(alldata_pd.values))
print("\t\t max(|pd.Dataframe.values - np.array|)=",np.max(np.abs(alldata_pd.values-alldata_np)))

In [ ]:
# Visualisation du dataframe dans le notebook
alldata_pd

### Mise en forme

Les données que nous avons contiennent les TN et les TX mais nous souhaitons faire une classification seulement sur les TN.
De plus, pour faire une classification, on mettre les données dans une matrice de profil `(n_individus, n_predicteurs)`.
Pour les deux variables issues de l'extraction, `alldata_np` et `alldata_pd`, il est possible d'extraire les TN au format voulu en une seule ligne.

En sélectionnant judicieusement les indices (slicing), extraire les séries temporelles de Tn pour toutes les stations dans une matrice nommée `TN`. S'assurer qu'elle est bien de profil `(n_individus, n_predicteurs)`.

In [ ]:
# Mise en forme : extraction des TN dans une matrice (n_individus, n_predicteurs)

# Depuis Numpy
TN=######
# Depuis Pandas
TN=######

N,p = np.shape(TN)
print("On dispose maintenant d'un jeu de données avec ",N,"individus et ",p,"prédicteurs")

In [ ]:
# BONUS : extraire la matrice TN dans un dataframe et non un numpy.array

######

In [ ]:
# SUPER BONUS : créer un dataframe qui
# contienne la matrice TN (avec les colonnes correctement nommées)
# et indexée par la date (et non entier qui ressemble à une date)

######

Combien y a-t-il de données manquantes ?

In [ ]:
# Nombre de données manquantes
nb_missingVal = ######
print("Nombre de données manquantes=",nb_missingVal)

Classification hierarchique ascendante
--------------------------------------

In [ ]:
# Import du package scipy.cluster.hierarchy
from scipy.cluster import hierarchy as cha

### Critère de Ward

In [ ]:
# Calcul de la matrice de connexion ('linkage matrix') -> cha.linkage"

linkageMatrix=######

print("Retourne un ",type(linkageMatrix)," de taille ",np.shape(linkageMatrix))

In [ ]:
figsize = (8,8)

In [ ]:
# Tracé du dendrogramme -> cha.dendrogram

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure(figsize=figsize)
plt.title(u"Dendrogramme (critère de Ward)")
######
plt.ylabel(u"Distance cophénétique")
plt.xlabel("Stations")
plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [ ]:
# Choix de K par examen de la distance cophénétique (3e colonne de la matrice de connexion)

distance_cophenetique = ######

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure(figsize=figsize)
plt.title(u"Croissance de la distance cophénétique (Ward)")
plt.bar(np.arange(linkageMatrix.shape[0],0,-1),distance_cophenetique,0.5)
plt.gca().invert_xaxis()
plt.ylabel(u"Distance cophénétique")
plt.xlabel("Nombre de classes")
plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

**?** D’après ces deux graphiques, quels nombres de classes semblent pertinents ?

In [ ]:
# Formation des groupes -> cha.fcluster

K=######
labels_ward=######

### Critère du saut minimum

Répétez les même opérations que pour le critère de Ward. Nous allons ensuite comparer les résultats.

In [ ]:
# Calcul de la matrice de connexion ('linkage matrix') -> cha.linkage"

linkageMatrix=######

In [ ]:
# Tracé du dendrogramme-> cha.dendrogram

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure(figsize=(10,10))
plt.title(u"Dendrogramme (saut minimum)")
######
plt.ylabel(u"Distance cophénétique")
plt.xlabel("Stations")
plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

In [ ]:
# Choix de K par examen de la distance cophénétique (3e colonne de la matrice de connexion)

distance_cophenetique = ######

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure(figsize=figsize)
plt.title(u"Croissance de la distance cophénétique (saut minimum)")
plt.bar(np.arange(linkageMatrix.shape[0],0,-1),distance_cophenetique,0.5)
plt.gca().invert_xaxis()
plt.ylabel(u"Distance cophénétique")
plt.xlabel("Nombre de classes")
plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

**?** Quelles différences observez-vous
  * sur le dendrogramme ;
  * sur la distance cophénétique ;
  * sur le nombre de classes à retenir ?

**?** Quel critère (entre Ward et saut minimum) vous semble le plus approprié ?

Visualisation des classes
-----------------------------

Nous avons maintenant une classification des stations suivant leur température minimale. Pour la commenter, il est important de pouvoir se représenter correctement ces classes. Nous cherchons ici à identifier des climats différents selon la région. On va donc regarder d’une part le cycle annuel des Tn pour le point moyen de chaque cluster et d’autre part positionner les stations avec leur classe sur une carte.

In [ ]:
labels = labels_ward

### Cycle annuel

In [ ]:
# Definition de la colormap (permet d'attribuer toujours la même couleur dans différents graphiques)
colmap=cm.nipy_spectral(np.arange(K)/K)

In [ ]:
# Calcul des moyennes mensuelles -> pandas.groupby
mois_en3lettres = ['Jan','Fev','Mar','Avr','Mai','Jun','Juil','Aou','Sep','Oct','Nov','Dec']

TN_pd = cp.solution_bonus1(alldata_pd)
moyenne_mensuelle = ######

In [ ]:
# Calculer les moyennes mensuelles par cluster
moyenne_mensuelle_par_cluster = np.zeros((K,12))
######

In [ ]:
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure(figsize=figsize)
plt.title("Cycle annuel pour chaque cluster")
for k in range(K):
    plt.plot(
        mois_en3lettres,
        moyenne_mensuelle_par_cluster[k,:],
        color=colmap[k],
        label='Cluster '+str(k+1),
        linewidth=2
    )
plt.ylabel("TN")
plt.xlabel("Mois")
plt.legend(loc='best')
plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

### Sur une carte

In [ ]:
# Extraction des coordonnées des stations

coordFile="../data/coord-83stations_points.txt"

# Avec Pandas
coords = ######

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt

In [ ]:
# Tracé de la carte...

stamen_terrain = cimgt.StamenTerrain()
minlon,maxlon,minlat,maxlat = (-6.3, 11.5, 41.8, 51.2)

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection=stamen_terrain.crs)
ax.set_extent([minlon,maxlon,minlat,maxlat], crs=ccrs.Geodetic())

# Add the Stamen data. Start at zoom level 3, then try 6.
ax.add_image(stamen_terrain, 6)

resolution = '50m'
ax.add_feature(
    cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', resolution, edgecolor='black', facecolor='none')
)

ax.scatter(
    coords.lon,
    coords.lat,
    marker='o',
    c=cm.nipy_spectral((labels-1)/K),
    s=60,
    alpha=0.7,
    transform=ccrs.Geodetic()
)

plt.show(block=False)
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

CLASSIFICATION EN PARTITIONS
======================================================================

4.1 K-means
---------------------------------------------------
Il existe deux packages pour utiliser les K-means : `pyclustering` et `scikit-learn`

In [ ]:
from pyclustering.cluster.kmeans import kmeans

In [ ]:
# On prend K points de l'echantillon que l'on perturbe
init_medoids=TN[np.random.choice(N,K,replace=False),:]+2*np.random.sample(p)-1

# Créer une instance de Kmeans
km_pyclust = ######

# Effectuer les calculs
######

# Récupérer les clusters
clusters_km_pyclust = ######
print("Kmeans (pyclustering) retourne ",type(clusters_km_pyclust),"de taille",len(clusters_km_pyclust))

# Transformation en labels (à la main)
labels_km_pyclust = np.zeros(N)
######

In [ ]:
from sklearn.cluster import KMeans

n_init=20

# Créer une instance de Kmeans
km_spy = ######

# Effectuer les calculs
######

# Récupérer les labels
labels_km_spy = ######
print("Kmeans (scipy) retourne ",type(labels_km_spy),"de taille",len(labels_km_spy))

4.2 K-medoids
--------------------------------------------------
Les K-medoids ne sont pas (encore) disponible dans `scikit-learn`.
Ils le sont dans une extension non-officielle et dans `pyclustering`.

In [ ]:
from pyclustering.cluster.kmedoids import kmedoids

In [ ]:
# Choix des points de départ (aléatoire)
init_medoids=np.random.choice(N,K,replace=False)
print(u"Choix des points de départ (aléatoire) : ",K,"points de l'échantillon. Indices :",init_medoids)

# Créer une instance de Kmedoids
pam = ######

# Effectuer les calculs
######

# Récupérer les clusters
clusters_pam = ######

print("Kmedoids (pyclustering) retourne ",type(clusters_pam),"de taille",len(clusters_pam))

# Transformation en labels
labels_pam = np.zeros(N)
######

4.3 Calcul des silhouettes
--------------------------------------------------

In [ ]:
labels = labels_pam

In [ ]:
# Fonction pour le tracé des silhouettes (donnée)
cp.plot_silhouette

In [ ]:
from sklearn import metrics

In [ ]:
# Calcul du score de silhouette -> sklearn.metrics.silhouette_score
silhouette_values = ######
print("Retourne un",type(silhouette_values)," de taille ",silhouette_values.shape)

In [ ]:
# Calcul du score de silhouette -> sklearn.metrics.silhouette_score
sil_score = ######
print("Retourne un",type(sil_score)," de valeur ",sil_score)

In [ ]:
cp.plot_silhouette(silhouette_values, labels)

Pour visualiser les clusters, réexécuter les cellules dédiées avec `labels=labels_km_spy` (par exemple).

Deverrouillage de la correction
--------------------------------------------
Pour déverrouiller le notebook qui contient la correction, il vous faudra utiliser la commande `gpg correction.ipynb`.
Une phrase de passe vous sera demandée.
Il s'agit du score de silhouette obtenu avec le critère de Ward, avec 6 décimales...

In [ ]:
######